## Import Libraries ##

In [1]:
import os
import math
import pandas as pd
import shapely
import numpy as np
import rasterio
from rasterio.transform import from_origin
import geoio
from osgeo import gdal, osr, ogr
import geopandas as gpd
import shapefile as shp
from shapely.geometry import Point, Polygon
from scipy.interpolate import griddata
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
from tqdm import tqdm
from rasterio.features import geometry_window, geometry_mask, shapes
from rasterio.warp import reproject, Resampling
from rasterio.enums import Resampling

## Define Filepaths ##

In [7]:
BASE_DIR = 'C:\\Users\\chris\\Desktop\\use-of-AI-to-eradicate-extreme-poverty'
SETTLEMENT_DIRS = [
    os.path.join(BASE_DIR, 'data/GlobalHumanSettlement/GHS_BUILT_S_E2020_GLOBE_R2023A_54009_100_V1_0_R11_C22.tif'),
    os.path.join(BASE_DIR, 'data/GlobalHumanSettlement/GHS_BUILT_S_E2020_GLOBE_R2023A_54009_100_V1_0_R12_C22.tif')
]
NIGHTLIGHTS_DIR = os.path.join(BASE_DIR, 'data', 'nightlights', 'viirs_2020_00N060W.tif')
ROADS_DIR = os.path.join(BASE_DIR, 'data', 'roads', 'GRIP4_density_total','grip4_total_dens_m_km2.asc')
HEALTH_DIR = os.path.join(BASE_DIR, 'data', 'health_facilities', 'healthcare_2020.shp')
LSMS_DIR = os.path.join(BASE_DIR, 'data', 'LSMS_2019')
csv_file_path = os.path.join(LSMS_DIR, 'HouseholdGeovariables_csv', 'householdgeovariables_ihs5.csv')

## Raster Reprojection and Merging ##

### Roads ###

In [3]:
#function to reproject road density data

def reproject_raster(input_tif, output_tif, target_epsg=4326):
    # Open the source dataset
    src_ds = gdal.Open(input_tif)
    if src_ds is None:
        raise RuntimeError("Unable to open the input file")

    # Define the target spatial reference using EPSG code
    target_srs = osr.SpatialReference()
    target_srs.ImportFromEPSG(target_epsg)

    # Get the source geotransform and spatial reference
    src_gt = src_ds.GetGeoTransform()
    src_srs = src_ds.GetProjectionRef()

    # Define the destination geotransform and spatial reference
    dst_gt = list(src_gt)
    dst_srs = target_srs.ExportToWkt()

    # Perform the reprojection
    dst_ds = gdal.Warp(output_tif, src_ds, dstSRS=dst_srs, format='GTiff')
    if dst_ds is None:
        raise RuntimeError("Reprojection failed.")
    
    # Clean up
    dst_ds = None
    src_ds = None
    print(f"Reprojection complete. Output saved at {output_tif}")

# Use the function
ROADS_DIR = os.path.join(BASE_DIR, 'data', 'roads', 'GRIP4_density_total', 'grip4_total_dens_m_km2.asc')
output_road_density_tif = os.path.join(BASE_DIR, 'data', 'roads', 'GRIP4_density_total', 'grip4_total_dens_m_km2_4326.tif')
reproject_raster(ROADS_DIR, output_road_density_tif)

Reprojection complete. Output saved at C:\Users\chris\Desktop\use-of-AI-to-eradicate-extreme-poverty\data\roads\GRIP4_density_total\grip4_total_dens_m_km2_4326.tif


In [8]:
df = pd.read_csv(csv_file_path) ## to read  the LSMS data csvs

### Builtup areas ###

In [10]:
#function to merge and reproject built up areas data
def merge_and_reproject(input_tifs, output_tif):
    # create a VRT to  merge TIFFs
    vrt_options = gdal.BuildVRTOptions(resampleAlg=gdal.GRA_NearestNeighbour, separate=False)
    vrt = gdal.BuildVRT("/vsimem/temp.vrt", input_tifs, options=vrt_options)
    if vrt is None:
        raise RuntimeError("Failed to create a virtual raster for merging.")

    # define the target spatial reference (WGS84)
    target_srs = osr.SpatialReference()
    target_srs.ImportFromEPSG(4326)

    # reprject the virtual merged raster
    warp_options = gdal.WarpOptions(dstSRS=target_srs.ExportToWkt(), resampleAlg=gdal.GRA_NearestNeighbour)
    result_ds = gdal.Warp(output_tif, vrt, options=warp_options)
    if result_ds is None:
        raise RuntimeError("Reprojection failed.")

    result_ds = None
    vrt = None
    print(f"Merge and reprojection complete. Output saved at {output_tif}")

# define the base directory
BASE_DIR = r'C:\Users\chris\Desktop\use-of-AI-to-eradicate-extreme-poverty'

# predefined paths for input TIFFs
SETTLEMENT_DIRS = [
    os.path.join(BASE_DIR, 'data', 'GlobalHumanSettlement', 'GHS_BUILT_S_E2020_GLOBE_R2023A_54009_100_V1_0_R11_C22.tif'),
    os.path.join(BASE_DIR, 'data', 'GlobalHumanSettlement', 'GHS_BUILT_S_E2020_GLOBE_R2023A_54009_100_V1_0_R12_C22.tif')
]

# define the output path for the merged and reprojected TIFF
output_tif = os.path.join(BASE_DIR, 'results', 'rasters', 'settlements_merged.tif')


output_dir = os.path.dirname(output_tif)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# execute the function
merge_and_reproject(SETTLEMENT_DIRS, output_tif)

Merge and reprojection complete. Output saved at C:\Users\chris\Desktop\use-of-AI-to-eradicate-extreme-poverty\results\rasters\settlements_merged.tif


### Health facilities ###

In [ ]:
## add heatmap function ##